# Working with Cellmage in Jupyter Notebooks

This notebook demonstrates how to interact with Cellmage directly within Jupyter notebooks using its API.

**Date:** April 24, 2025

## Setup and Configuration

Let's start by setting up our environment and importing the necessary modules.

In [1]:
# Add the parent directory to sys.path so we can import cellmage
import os
import sys
import logging
from IPython.display import display, Markdown

# Set up logging
logging.basicConfig(level=logging.INFO)

# Ensure the cellmage package can be imported
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))

print(f"Notebook directory: {notebook_dir}")
print(f"Project root directory: {project_root}")

if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added path: {project_root}")

try:
    # Import cellmage
    import cellmage

    print(f"Cellmage version: {cellmage.__version__}")
except Exception as e:
    print(f"Error importing cellmage: {e}")
    print("\nDebug information:")
    print(f"Current working directory: {os.getcwd()}")
    print(f"Python path: {sys.path}")

2025-04-24 06:21:25,341 - cellmage - INFO - Cellmage logging initialized
2025-04-24 06:21:25,344 - cellmage.config - INFO - python-dotenv not installed, skipping .env file loading
2025-04-24 06:21:25,344 - cellmage.config - INFO - python-dotenv not installed, skipping .env file loading


Notebook directory: /Users/tpinto/madpin/cellmage/notebooks
Project root directory: /Users/tpinto/madpin/cellmage
Added path: /Users/tpinto/madpin/cellmage
Cellmage version: 0.1.0


## Understanding the Cellmage Architecture

Cellmage has a modular architecture with several key components:

1. **LLM Client**: Handles communication with language model providers
2. **Persona Loader**: Manages system prompts and configurations for different personas
3. **Snippet Provider**: Manages code snippets that can be added to conversations
4. **History Store**: Stores conversation history
5. **Chat Manager**: The central component that coordinates all of the above

Let's set up these components for a basic workflow.

## Setting Up the LLM Client

First, we'll set up the LLM client. Cellmage provides the `DirectLLMAdapter` which can connect to OpenAI, Google, Anthropic, and other LLM providers.

In [2]:
from cellmage.adapters.direct_client import DirectLLMAdapter

# Create a DirectLLMAdapter with your preferred model
# You'll need to ensure your API key is set in your environment variables
# For example: OPENAI_API_KEY, GOOGLE_API_KEY, etc.
llm_client = DirectLLMAdapter(default_model="gpt-4.1-nano")

print("LLM client initialized.")

# Optional: Set parameter overrides
llm_client.set_override("temperature", 0.7)
print(f"Temperature override set to 0.7")

2025-04-24 06:21:25,405 - cellmage.adapters.direct_client - INFO - [Override] Setting 'api_key' = sk-L...xxmA
2025-04-24 06:21:25,405 - cellmage.adapters.direct_client - INFO - [Override] Setting 'api_base' = https://litellm.oracle.madpin.dev
2025-04-24 06:21:25,406 - cellmage.adapters.direct_client - INFO - [Override] Setting 'model' = gpt-4.1-nano
2025-04-24 06:21:25,406 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.7
2025-04-24 06:21:25,405 - cellmage.adapters.direct_client - INFO - [Override] Setting 'api_base' = https://litellm.oracle.madpin.dev
2025-04-24 06:21:25,406 - cellmage.adapters.direct_client - INFO - [Override] Setting 'model' = gpt-4.1-nano
2025-04-24 06:21:25,406 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.7


LLM client initialized.
Temperature override set to 0.7


## Creating Personas

Personas define the behavior of the LLM through system prompts and parameter configurations.

In [3]:
from cellmage.resources.memory_loader import MemoryLoader

# Create a memory-based persona loader
persona_loader = MemoryLoader()

# Create a persona for Python code assistance
persona_loader.add_persona(
    name="python_expert",
    system_message="You are a Python programming expert who provides clean, efficient code examples with clear explanations. Focus on best practices and modern Python features.",
    config={"temperature": 0.3},
)

# Create a persona for creative writing
persona_loader.add_persona(
    name="creative_writer",
    system_message="You are a creative writing assistant who helps craft engaging stories and narratives. Be imaginative and provide vivid, descriptive content.",
    config={"temperature": 0.8},
)

# List available personas
personas = persona_loader.list_personas()
print(f"Available personas: {personas}")

2025-04-24 06:21:25,446 - cellmage.resources.memory_loader - INFO - Added persona 'python_expert' to memory
2025-04-24 06:21:25,447 - cellmage.resources.memory_loader - INFO - Added persona 'creative_writer' to memory
2025-04-24 06:21:25,447 - cellmage.resources.memory_loader - INFO - Added persona 'creative_writer' to memory


Available personas: ['creative_writer', 'python_expert']


## Managing Code Snippets

Snippets are useful for providing additional context to the LLM. We can reuse our persona loader to manage snippets.

In [4]:
# We can use the same MemoryLoader for snippets
snippet_provider = MemoryLoader()

# Create a sample code snippet
snippet_provider.add_snippet(
    name="fibonacci_recursive",
    content="""```python
def calculate_fibonacci(n):
    ""Calculate the nth Fibonacci number.""
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return calculate_fibonacci(n-1) + calculate_fibonacci(n-2)
    
# Test the function
result = calculate_fibonacci(10)
print(f"The 10th Fibonacci number is: {result}")
```""",
)

# List available snippets
snippets = snippet_provider.list_snippets()
print(f"Available snippets: {snippets}")

2025-04-24 06:21:25,453 - cellmage.resources.memory_loader - INFO - Added snippet 'fibonacci_recursive' to memory


Available snippets: ['fibonacci_recursive']


## Setting Up the Chat Manager

Now we'll set up the ChatManager which coordinates all the components.

In [5]:
from cellmage.storage.memory_store import MemoryStore

# Create a memory-based history store
history_store = MemoryStore()

# Create the chat manager
chat_manager = cellmage.ChatManager(
    llm_client=llm_client,
    persona_loader=persona_loader,
    snippet_provider=snippet_provider,
    history_store=history_store,
)

# Set default persona
chat_manager.set_default_persona("python_expert")
print("Chat manager initialized with 'python_expert' persona.")

2025-04-24 06:21:25,462 - cellmage.chat_manager - INFO - Initializing ChatManager
2025-04-24 06:21:25,463 - cellmage.config - INFO - python-dotenv not installed, skipping .env file loading
2025-04-24 06:21:25,463 - cellmage.chat_manager - INFO - ChatManager initialized
2025-04-24 06:21:25,463 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.3
2025-04-24 06:21:25,464 - cellmage.chat_manager - INFO - Default persona set to 'python_expert'
2025-04-24 06:21:25,463 - cellmage.config - INFO - python-dotenv not installed, skipping .env file loading
2025-04-24 06:21:25,463 - cellmage.chat_manager - INFO - ChatManager initialized
2025-04-24 06:21:25,463 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.3
2025-04-24 06:21:25,464 - cellmage.chat_manager - INFO - Default persona set to 'python_expert'


Chat manager initialized with 'python_expert' persona.


## Basic LLM Interaction

Now that we have all components set up, let's interact with the LLM.

In [6]:
# Display function for better visualization of responses
def display_response(content):
    display(Markdown(f"**Assistant:**\n{content}"))


# Send a message to the LLM
prompt = "Explain the concept of list comprehensions in Python with a few examples."
print(f"User: {prompt}")

# Get response (stream=False for simplicity)
response = chat_manager.chat(prompt, stream=False)

# Display the response
display_response(response)

2025-04-24 06:21:25,541 - cellmage.chat_manager - INFO - Sending message to LLM with 2 messages in context


User: Explain the concept of list comprehensions in Python with a few examples.


2025-04-24 06:21:25,549 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 2 messages


**Assistant:**
Certainly! 

**List comprehensions** in Python provide a concise way to create lists. They allow you to generate new lists by applying an expression to each item in an existing iterable (like a list, range, etc.), optionally filtering items with a condition.

### Basic syntax:
```python
[expression for item in iterable if condition]
```

- `expression`: The value to store in the new list.
- `item`: The variable representing each element in the iterable.
- `iterable`: The collection you're iterating over.
- `condition` (optional): A predicate that filters items.

---

### Examples

#### 1. Creating a list of squares
```python
squares = [x**2 for x in range(5)]
print(squares)  # Output: [0, 1, 4, 9, 16]
```

#### 2. Filtering items
Suppose you want only even numbers from 0 to 9:
```python
evens = [x for x in range(10) if x % 2 == 0]
print(evens)  # Output: [0, 2, 4, 6, 8]
```

#### 3. Applying a function
Convert a list of strings to their uppercase versions:
```python
words = ['apple', 'banana', 'cherry']
uppercase_words = [word.upper() for word in words]
print(uppercase_words)  # Output: ['APPLE', 'BANANA', 'CHERRY']
```

#### 4. Nested list comprehensions
Create a multiplication table (e.g., 1 to 3):
```python
table = [[i * j for j in range(1, 4)] for i in range(1, 4)]
print(table)
# Output: [[1, 2, 3], [2, 4, 6], [3, 6, 9]]
```

---

### Benefits
- More concise and readable than traditional loops.
- Often faster due to internal optimizations.

### Summary
List comprehensions are a powerful feature for creating and transforming lists efficiently and cleanly in Python.

## Using a Different Persona

We can switch personas for different types of interactions.

In [7]:
# Change the active persona
chat_manager.set_default_persona("creative_writer")
print("Switched to 'creative_writer' persona.")

# Send a creative writing prompt
creative_prompt = "Write a short poem about coding in Python."
print(f"User: {creative_prompt}")

# Get response
creative_response = chat_manager.chat(creative_prompt, stream=False)

# Display the response
display_response(creative_response)

2025-04-24 06:21:28,758 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.8
2025-04-24 06:21:28,758 - cellmage.chat_manager - INFO - Default persona set to 'creative_writer'
2025-04-24 06:21:28,759 - cellmage.chat_manager - INFO - Sending message to LLM with 4 messages in context
2025-04-24 06:21:28,760 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 4 messages
2025-04-24 06:21:28,758 - cellmage.chat_manager - INFO - Default persona set to 'creative_writer'
2025-04-24 06:21:28,759 - cellmage.chat_manager - INFO - Sending message to LLM with 4 messages in context
2025-04-24 06:21:28,760 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 4 messages


Switched to 'creative_writer' persona.
User: Write a short poem about coding in Python.


**Assistant:**
In lines of Python, magic starts,  
A world of logic, crafted with arts.  
Indentations dance, clean and bright,  
Turning thoughts to code in gentle light.

Variables whisper, functions sing,  
Loops and conditions in harmony bring,  
A tapestry woven with every line,  
Python's poetry, elegant and fine.

## Using Snippets for Context

We can add snippets to provide context before our query.

In [8]:
# Switch back to python_expert persona
chat_manager.set_default_persona("python_expert")
print("Switched to 'python_expert' persona.")

# Clear history for a clean conversation
chat_manager.clear_history()
print("History cleared.")

# Add the snippet to the conversation
chat_manager.add_snippet("fibonacci_recursive")
print("Added 'fibonacci_recursive' snippet to the conversation.")

# Send a query about the snippet
snippet_query = "The recursive Fibonacci implementation is inefficient. Please provide an optimized version using dynamic programming and explain the performance improvement."
print(f"User: {snippet_query}")

# Get response
snippet_response = chat_manager.chat(snippet_query, stream=False)

# Display the response
display_response(snippet_response)

2025-04-24 06:21:29,636 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.3
2025-04-24 06:21:29,636 - cellmage.chat_manager - INFO - Default persona set to 'python_expert'
2025-04-24 06:21:29,637 - cellmage.history_manager - INFO - History cleared. Kept 1 system messages.
2025-04-24 06:21:29,638 - cellmage.chat_manager - INFO - Added snippet 'fibonacci_recursive' as system message
2025-04-24 06:21:29,638 - cellmage.chat_manager - INFO - Sending message to LLM with 3 messages in context
2025-04-24 06:21:29,638 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 3 messages
2025-04-24 06:21:29,636 - cellmage.chat_manager - INFO - Default persona set to 'python_expert'
2025-04-24 06:21:29,637 - cellmage.history_manager - INFO - History cleared. Kept 1 system messages.
2025-04-24 06:21:29,638 - cellmage.chat_manager - INFO - Added snippet 'fibonacci_recursive' as system message
2025-04-24 06:21:29,638 - cellmage.chat_manager - INFO - Se

Switched to 'python_expert' persona.
History cleared.
Added 'fibonacci_recursive' snippet to the conversation.
User: The recursive Fibonacci implementation is inefficient. Please provide an optimized version using dynamic programming and explain the performance improvement.


**Assistant:**
Certainly! The recursive implementation of Fibonacci has exponential time complexity because it recalculates the same subproblems multiple times. To optimize this, we can use **dynamic programming**—either with memoization (top-down) or tabulation (bottom-up)—to store previously computed results and avoid redundant calculations.

Here's an efficient, iterative (bottom-up) implementation using a list to store intermediate results:

```python
def calculate_fibonacci(n):
    """Calculate the nth Fibonacci number using dynamic programming (bottom-up approach)."""
    if n <= 0:
        return 0
    elif n == 1:
        return 1

    fib = [0] * (n + 1)
    fib[1] = 1

    for i in range(2, n + 1):
        fib[i] = fib[i - 1] + fib[i - 2]

    return fib[n]

# Test the function
result = calculate_fibonacci(10)
print(f"The 10th Fibonacci number is: {result}")
```

### Performance Improvement:
- **Time Complexity:** O(n), since each Fibonacci number up to `n` is computed exactly once.
- **Space Complexity:** O(n), due to the list storing all intermediate Fibonacci numbers. This can be optimized further to O(1) space if needed.

### Optimized Space Version:
If you only need the `n`th Fibonacci number without storing the entire sequence:

```python
def calculate_fibonacci(n):
    """Calculate the nth Fibonacci number using constant space."""
    if n <= 0:
        return 0
    elif n == 1:
        return 1

    prev, curr = 0, 1
    for _ in range(2, n + 1):
        prev, curr = curr, prev + curr

    return curr

# Test the function
result = calculate_fibonacci(10)
print(f"The 10th Fibonacci number is: {result}")
```

This version maintains only two variables, making it more space-efficient while still running in linear time.

## Adjusting Parameters for a Single Query

We can override parameters for specific queries using the `chat()` method directly.

In [9]:
# Clear history for a clean conversation
chat_manager.clear_history()
print("History cleared.")

# Send a query with a high temperature for more creative responses
custom_query = "Generate three creative ways to explain recursion to a 10-year-old."
print(f"User: {custom_query}")

# Get response with a custom temperature
custom_response = chat_manager.chat(
    custom_query,
    stream=False,
    temperature=0.9,  # Override temperature just for this query
)

# Display the response
display_response(custom_response)

2025-04-24 06:21:32,357 - cellmage.history_manager - INFO - History cleared. Kept 2 system messages.
2025-04-24 06:21:32,358 - cellmage.chat_manager - INFO - Sending message to LLM with 3 messages in context
2025-04-24 06:21:32,358 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 3 messages
2025-04-24 06:21:32,358 - cellmage.chat_manager - INFO - Sending message to LLM with 3 messages in context
2025-04-24 06:21:32,358 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 3 messages


History cleared.
User: Generate three creative ways to explain recursion to a 10-year-old.


**Assistant:**
Sure! Here are three fun and creative ways to explain recursion to a 10-year-old:

### 1. **The Russian Nesting Dolls**
Imagine you have a set of beautiful dolls called nesting dolls. When you open one doll, you find a smaller doll inside. If you open that smaller doll, there's an even smaller one inside, and so on, until you reach the tiniest doll which doesn't open at all.

**Recursion** is like opening these dolls: **each time you open a doll, you're doing the same action (opening), but with a smaller doll**. The process continues until you reach the smallest doll, which is the **base case**—the point where you stop opening more dolls.

---

### 2. **The Infinite Mirror Reflection**
Picture standing between two mirrors facing each other. When you look into the mirrors, you see a series of your reflection repeating over and over again, each one smaller than the last.

**Recursion** is like that: a process that repeats itself, each time getting closer to a final, smallest image (the base case). When the reflection gets too small, you decide to stop, just like when a recursive function stops calling itself.

---

### 3. **Climbing Down a Stairs with a Puzzle**
Imagine you're climbing down a staircase, and at each step, there's a small puzzle you need to solve before you can go down to the next step.

**Recursion** is like solving one small puzzle at each step: you solve the first puzzle, then move to the next. When you reach the bottom step (the base case), you're done. The process of solving each puzzle is the same, just like the recursive function calls itself at a smaller problem each time.

---

Would you like a simple code example to go with one of these explanations?

## Managing Conversation History

Let's examine the conversation history and how to manage it.

In [10]:
# Get the current history
history = chat_manager.get_history()
print(f"Current history has {len(history)} messages.")

# Display each message in the history
for i, message in enumerate(history):
    role = message.role.upper()
    content_preview = message.content[:50] + "..." if len(message.content) > 50 else message.content
    print(f"{i + 1}. {role}: {content_preview}")

Current history has 4 messages.
1. SYSTEM: You are a Python programming expert who provides c...
2. SYSTEM: ```python
def calculate_fibonacci(n):
    ""Calcul...
3. USER: Generate three creative ways to explain recursion ...
4. ASSISTANT: Sure! Here are three fun and creative ways to expl...


## Multi-Turn Conversation

Let's have a multi-turn conversation to test the context retention.

In [11]:
# Clear history for a clean conversation
chat_manager.clear_history()
print("History cleared.\n")

# First turn
print("--- First Turn ---")
print("User: What is a decorator in Python?")
response1 = chat_manager.chat("What is a decorator in Python?", stream=False)
display_response(response1)

# Second turn
print("\n--- Second Turn ---")
print("User: Can you show me a simple example of a timing decorator?")
response2 = chat_manager.chat("Can you show me a simple example of a timing decorator?", stream=False)
display_response(response2)

# Third turn
print("\n--- Third Turn ---")
print("User: How would I modify this decorator to also log the function name?")
response3 = chat_manager.chat("How would I modify this decorator to also log the function name?", stream=False)
display_response(response3)

2025-04-24 06:21:34,695 - cellmage.history_manager - INFO - History cleared. Kept 2 system messages.
2025-04-24 06:21:34,696 - cellmage.chat_manager - INFO - Sending message to LLM with 3 messages in context
2025-04-24 06:21:34,698 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 3 messages
2025-04-24 06:21:34,696 - cellmage.chat_manager - INFO - Sending message to LLM with 3 messages in context
2025-04-24 06:21:34,698 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 3 messages


History cleared.

--- First Turn ---
User: What is a decorator in Python?


**Assistant:**
A decorator in Python is a design pattern that allows you to modify or enhance the behavior of functions or classes **without changing their actual code**. Decorators are applied using the `@decorator_name` syntax above a function or class definition.

**In simple terms:**
- Think of a decorator as a wrapper around a function.
- It takes a function as input, adds some functionality, and returns a new function.

**Example:**

```python
def my_decorator(func):
    def wrapper():
        print("Before the function runs")
        func()
        print("After the function runs")
    return wrapper

@my_decorator
def say_hello():
    print("Hello!")

say_hello()
```

**Output:**
```
Before the function runs
Hello!
After the function runs
```

**Key points:**
- Decorators are often used for logging, access control, timing, or modifying input/output.
- They are a powerful feature for writing clean, reusable, and expressive code.

2025-04-24 06:21:37,968 - cellmage.chat_manager - INFO - Sending message to LLM with 5 messages in context
2025-04-24 06:21:37,969 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 5 messages
2025-04-24 06:21:37,969 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 5 messages



--- Second Turn ---
User: Can you show me a simple example of a timing decorator?


**Assistant:**
Certainly! Here's a simple example of a timing decorator that measures how long a function takes to execute:

```python
import time

def timing_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()  # Record start time
        result = func(*args, **kwargs)  # Call the original function
        end_time = time.time()  # Record end time
        duration = end_time - start_time
        print(f"{func.__name__} took {duration:.4f} seconds to execute.")
        return result
    return wrapper

@timing_decorator
def example_function(n):
    total = 0
    for i in range(n):
        total += i
    return total

# Call the decorated function
result = example_function(10**6)
print(f"Result: {result}")
```

**Explanation:**
- The `timing_decorator` wraps any function to measure its execution time.
- `*args` and `**kwargs` allow it to work with functions of any signature.
- It prints the duration after the function completes.

**When you run this code, you'll see output similar to:**

```
example_function took 0.0456 seconds to execute.
Result: 499999500000
```

2025-04-24 06:21:43,324 - cellmage.chat_manager - INFO - Sending message to LLM with 7 messages in context
2025-04-24 06:21:43,326 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 7 messages
2025-04-24 06:21:43,326 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 7 messages



--- Third Turn ---
User: How would I modify this decorator to also log the function name?


**Assistant:**
To include the function name in the log, you can access `func.__name__` inside the decorator. Here's the modified version:

```python
import time

def timing_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        duration = end_time - start_time
        print(f"Function '{func.__name__}' took {duration:.4f} seconds to execute.")
        return result
    return wrapper

@timing_decorator
def example_function(n):
    total = 0
    for i in range(n):
        total += i
    return total

# Call the decorated function
result = example_function(10**6)
print(f"Result: {result}")
```

**Output:**

```
Function 'example_function' took 0.0456 seconds to execute.
Result: 499999500000
```

**Summary:**
- Using `func.__name__` inside the decorator allows you to dynamically include the function's name in your logs, making it clearer which function's timing is being reported.

## Saving and Loading Conversations

Cellmage allows you to save and load conversations.

In [12]:
# Save the current conversation
save_path = chat_manager.save_conversation("decorator_conversation")
print(f"Conversation saved to: {save_path}")

# Clear history
chat_manager.clear_history()
print(f"History cleared, current message count: {len(chat_manager.get_history())}")

# Load the saved conversation
chat_manager.load_conversation(save_path)
print(f"Conversation loaded, message count: {len(chat_manager.get_history())}")

# Display the loaded conversation summary
print("\n=== Loaded Conversation Summary ===")
history = chat_manager.get_history()
for i, message in enumerate(history):
    if message.role != "system":
        role = message.role.upper()
        content_preview = message.content[:50] + "..." if len(message.content) > 50 else message.content
        print(f"{role}: {content_preview}")

2025-04-24 06:21:45,129 - cellmage.storage.memory_store - INFO - Saved conversation to memory with ID: decorator_conversation
2025-04-24 06:21:45,131 - cellmage.history_manager - INFO - History cleared. Kept 2 system messages.
2025-04-24 06:21:45,132 - cellmage.history_manager - INFO - Loaded conversation from decorator_conversation with 8 messages
2025-04-24 06:21:45,131 - cellmage.history_manager - INFO - History cleared. Kept 2 system messages.
2025-04-24 06:21:45,132 - cellmage.history_manager - INFO - Loaded conversation from decorator_conversation with 8 messages


Conversation saved to: decorator_conversation
History cleared, current message count: 2
Conversation loaded, message count: 8

=== Loaded Conversation Summary ===
USER: What is a decorator in Python?
ASSISTANT: A decorator in Python is a design pattern that all...
USER: Can you show me a simple example of a timing decor...
ASSISTANT: Certainly! Here's a simple example of a timing dec...
USER: How would I modify this decorator to also log the ...
ASSISTANT: To include the function name in the log, you can a...


## Parameter Overrides

We can set and manage parameter overrides for the LLM client.

In [13]:
# Set parameter overrides directly on the LLM client
llm_client.set_override("temperature", 0.5)
llm_client.set_override("max_tokens", 2000)
print("Parameter overrides set: temperature=0.5, max_tokens=2000")

# Check current overrides - this accesses the llm_client directly since
# ChatManager might not have a get_overrides method
# Try to get overrides safely
try:
    if hasattr(chat_manager, "get_overrides"):
        # Use the method if available
        overrides = chat_manager.get_overrides()
    else:
        # Access the overrides directly from the llm_client
        overrides = llm_client._overrides if hasattr(llm_client, "_overrides") else {}
    print(f"Current overrides: {overrides}")
except Exception as e:
    print(f"Couldn't retrieve overrides: {e}")

# Remove a specific override
llm_client.remove_override("max_tokens")
print("Removed 'max_tokens' override")

# Check updated overrides
try:
    overrides = llm_client._overrides if hasattr(llm_client, "_overrides") else {}
    print(f"Updated overrides: {overrides}")
except Exception as e:
    print(f"Couldn't retrieve overrides: {e}")

# Clear all overrides
llm_client.clear_overrides()
print("All overrides cleared")

# Check final overrides
try:
    overrides = llm_client._overrides if hasattr(llm_client, "_overrides") else {}
    print(f"Final overrides: {overrides}")
except Exception as e:
    print(f"Couldn't retrieve overrides: {e}")

2025-04-24 06:21:45,146 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.5
2025-04-24 06:21:45,149 - cellmage.adapters.direct_client - INFO - [Override] Setting 'max_tokens' = 2000
2025-04-24 06:21:45,150 - cellmage.adapters.direct_client - INFO - [Override] Removed 'max_tokens'
2025-04-24 06:21:45,152 - cellmage.adapters.direct_client - INFO - [Override] All instance overrides cleared.
2025-04-24 06:21:45,149 - cellmage.adapters.direct_client - INFO - [Override] Setting 'max_tokens' = 2000
2025-04-24 06:21:45,150 - cellmage.adapters.direct_client - INFO - [Override] Removed 'max_tokens'
2025-04-24 06:21:45,152 - cellmage.adapters.direct_client - INFO - [Override] All instance overrides cleared.


Parameter overrides set: temperature=0.5, max_tokens=2000
Current overrides: {}
Removed 'max_tokens' override
Updated overrides: {}
All overrides cleared
Final overrides: {}


## Handling Common Issues with Magic Functions

Here are some common issues you might encounter when using Cellmage's magic functions and how to resolve them:

1. **Magic command not found** - Make sure you've loaded the extension with `%load_ext cellmage`
2. **API connection errors** - Check your API key and connection settings
3. **Missing personas or snippets** - Use `--list-personas` or `--list-snippets` to check available resources
4. **Memory errors** - Try clearing history with `--clear-history` to free up memory
5. **Import errors** - Verify the cellmage package is correctly installed and in your Python path

## Conclusion

This notebook has demonstrated the key features of Cellmage's IPython magic functions, including:

1. Loading the cellmage extension
2. Managing personas and LLM parameters
3. Working with code snippets
4. Interacting with the LLM using `%%llm` cell magic
5. Managing conversation history
6. Saving and loading conversations
7. Understanding the underlying implementation

These magic functions make it easy to integrate LLMs directly into your notebook workflow, enhancing your productivity when working with AI assistants.